In [1]:
import numpy as np
import pandas as pd
import matplotlib as mql
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
from numpy.random import standard_normal as stn
from datetime import datetime
from sklearn import linear_model
import statsmodels.api as sm
import scipy.stats as scs
from scipy.stats import norm
import math
from cvxopt import matrix, solvers
import scipy.optimize as sco
from scipy.fftpack import fft,ifft
from sympy import *
from sklearn.decomposition import PCA
import json
from cvxopt import matrix, solvers
import scipy.optimize

E:\anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


## Bl model

先随便用几个ETF数据来做,下面是数据处理得到年化log return

In [2]:
icode=['AAPL','BA','TSLA','BABA','GS','GOOG','AMZN','TM','F','JPM']
data=pdr.get_data_yahoo(icode,start='2017-01-01',end='2019-01-01')   # For data
data_close=data['Adj Close']

icode2=['^IRX']
risk_free=pdr.get_data_yahoo(icode2,start='2017-01-01',end='2019-01-01')
risk_free=risk_free['Adj Close']/100

In [3]:
data_close.head()

Symbols,AAPL,BA,TSLA,BABA,GS,GOOG,AMZN,TM,F,JPM
Date,,,,,,,,,,
2017-01-03,110.691154,145.533676,216.990005,88.599998,229.168716,786.140015,753.669983,109.734962,10.415051,79.052376
2017-01-04,110.567276,147.063477,226.990005,90.510002,230.648605,786.900024,757.179993,112.178650,10.894855,79.198174
2017-01-05,111.129562,147.146927,226.750000,94.370003,228.931519,794.020020,780.450012,111.484436,10.563956,78.469170
2017-01-06,112.368469,147.508514,229.009995,93.889999,232.327744,806.150024,795.989990,111.197479,10.555683,78.478271
2017-01-09,113.397697,146.785309,231.279999,94.720001,230.420914,806.650024,796.919983,110.836479,10.448141,78.532944


In [4]:
returns = np.log(data_close.shift(-1)/data_close)
returns=returns.dropna(axis=0,how='all')     #kill nan
Cov=returns.cov()
annual_returns=returns*252

In [5]:
annual_returns.head()

Symbols,AAPL,BA,TSLA,BABA,GS,GOOG,AMZN,TM,F,JPM
Date,,,,,,,,,,
2017-01-03,-0.282180,2.635113,11.353783,5.374788,1.622093,0.243506,1.170896,5.550215,11.349750,0.464340
2017-01-04,1.278290,0.142955,-0.266591,10.524248,-1.883057,2.269882,7.627963,-1.564339,-7.772399,-2.330353
2017-01-05,2.793830,0.618486,2.499225,-1.285043,3.710990,3.820619,4.968411,-0.649475,-0.197430,0.029229
2017-01-06,2.297663,-1.238546,2.485588,2.217929,-2.076824,0.156250,0.294252,-0.819443,-2.580562,0.175496
2017-01-09,0.254038,1.190968,-1.541026,5.343697,-0.332206,-0.581755,-0.322735,0.042096,4.351754,0.729936


### BL model 建立
#### $\mu$

In [6]:
mu=np.mean(annual_returns)
mu

Symbols
AAPL    0.169469
BA      0.385788
TSLA    0.215126
BABA    0.219486
GS     -0.172359
GOOG    0.138631
AMZN    0.346854
TM      0.021575
F      -0.196012
JPM     0.082235
dtype: float64

#### $r_f$

In [7]:
rf=risk_free.mean()[0]
rf

0.014200682729243259

#### $\Sigma$

In [8]:
Cov=Cov*252

In [9]:
Cov.head()

Symbols,AAPL,BA,TSLA,BABA,GS,GOOG,AMZN,TM,F,JPM
Symbols,,,,,,,,,,
AAPL,0.056962,0.026320,0.035018,0.035597,0.025481,0.034471,0.043662,0.015030,0.012213,0.019482
BA,0.026320,0.064442,0.028154,0.031532,0.028079,0.028681,0.034109,0.017858,0.014637,0.023519
TSLA,0.035018,0.028154,0.229940,0.041285,0.023631,0.034706,0.045234,0.012347,0.023461,0.019655
BABA,0.035597,0.031532,0.041285,0.100194,0.027906,0.043376,0.051852,0.017723,0.018471,0.020923
GS,0.025481,0.028079,0.023631,0.027906,0.052992,0.027371,0.028061,0.017244,0.022015,0.035589


#### Investor-specific threshold level of risk L

In [10]:
L=1

#### $x_{mkt}$

In [11]:
mkt=pdr.get_quote_yahoo(icode)['marketCap']
mkt=mkt/(np.sum(mkt))

In [12]:
mkt

AAPL    0.268756
BA      0.016334
TSLA    0.031269
BABA    0.120941
GS      0.014179
GOOG    0.186617
AMZN    0.254360
TM      0.038819
F       0.004717
JPM     0.064008
Name: marketCap, dtype: float64

In [13]:
mu

Symbols
AAPL    0.169469
BA      0.385788
TSLA    0.215126
BABA    0.219486
GS     -0.172359
GOOG    0.138631
AMZN    0.346854
TM      0.021575
F      -0.196012
JPM     0.082235
dtype: float64

#### Directly $x_{hist}$

In [14]:
def opt1(mu,C,rf):
    
    e=np.zeros(num)+1
    
    def markowitz(weights,mu,C,rf):
    
        weights=np.array(weights)
    
        obj_func = np.dot(mu.T,weights) +  (1-np.dot(e.T,weights))*rf
    
        #print (-obj_func,weights)
        return -obj_func

        
    cons = ({'type':'eq', 'fun':lambda x: np.sum(x)-1},
       {'type':'ineq', 'fun':lambda x: -np.sqrt(np.dot(np.dot(x.T,C),x))+L})

    #No short constraint
    #bnds = [(0,1)]*num
    weights = np.ones(num)/num
    res = scipy.optimize.minimize(markowitz, weights,(mu,C,rf), method = 'SLSQP',  constraints = cons)
    print(res)
    if not res.success: raise BaseException(res.message)
    return res

In [15]:
num = 10
w = opt1(mu,Cov,rf).x *100
w = list(map(lambda x:round(x, 3), w))
print (w)

     fun: -3.031142351161039
     jac: array([-0.15526837, -0.37158778, -0.20092502, -0.20528576,  0.18656018,
       -0.12443   , -0.33265305, -0.00737458,  0.21021265, -0.06803393])
 message: 'Optimization terminated successfully.'
    nfev: 325
     nit: 27
    njev: 27
  status: 0
 success: True
       x: array([ 0.18646525,  2.34778699,  0.10030026,  0.29286346, -5.25907165,
       -0.49518406,  1.34622111, -0.26903583, -1.73543583,  4.4850903 ])
[18.647, 234.779, 10.03, 29.286, -525.907, -49.518, 134.622, -26.904, -173.544, 448.509]


In [16]:
sum(w)

99.99999999999989

#### BL model
$\delta$

In [17]:
delta=1.25

$P$

In [18]:
P=[-0.1,0,0,-0.2,0.4,-0.1,0.3,0,-0.4,0.1]
P=np.array(P)
P.sum()
P = -P

$\hat{\Sigma}$

In [19]:
Cov1=Cov

In [20]:
Cov1

Symbols,AAPL,BA,TSLA,BABA,GS,GOOG,AMZN,TM,F,JPM
Symbols,,,,,,,,,,
AAPL,0.056962,0.026320,0.035018,0.035597,0.025481,0.034471,0.043662,0.015030,0.012213,0.019482
BA,0.026320,0.064442,0.028154,0.031532,0.028079,0.028681,0.034109,0.017858,0.014637,0.023519
TSLA,0.035018,0.028154,0.229940,0.041285,0.023631,0.034706,0.045234,0.012347,0.023461,0.019655
BABA,0.035597,0.031532,0.041285,0.100194,0.027906,0.043376,0.051852,0.017723,0.018471,0.020923
GS,0.025481,0.028079,0.023631,0.027906,0.052992,0.027371,0.028061,0.017244,0.022015,0.035589
GOOG,0.034471,0.028681,0.034706,0.043376,0.027371,0.051729,0.048904,0.018814,0.015602,0.020515
AMZN,0.043662,0.034109,0.045234,0.051852,0.028061,0.048904,0.086385,0.017119,0.014872,0.020684
TM,0.015030,0.017858,0.012347,0.017723,0.017244,0.018814,0.017119,0.026472,0.014425,0.014592
F,0.012213,0.014637,0.023461,0.018471,0.022015,0.015602,0.014872,0.014425,0.053109,0.020259


$\hat{r}$

In [21]:
mkt

AAPL    0.268756
BA      0.016334
TSLA    0.031269
BABA    0.120941
GS      0.014179
GOOG    0.186617
AMZN    0.254360
TM      0.038819
F       0.004717
JPM     0.064008
Name: marketCap, dtype: float64

In [22]:
(2*delta*np.dot(Cov,mkt)).shape

(10,)

In [23]:
e = np.ones(10)
r_hat=rf*e+2*delta*np.dot(Cov,mkt)

In [24]:
e

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [25]:
r_hat

array([0.11651814, 0.08798639, 0.11975029, 0.13241186, 0.08287265,
       0.11585523, 0.14803261, 0.05670152, 0.05320767, 0.06778732])

$q$

In [26]:
q=np.dot(P.T,r_hat)
q=np.array([q])

In [27]:
q

array([-0.0133348])

$\Omega$

In [28]:
mm=1
r0=1/24
r1=0.02*r0
rr=[r1]

In [29]:
Cov

Symbols,AAPL,BA,TSLA,BABA,GS,GOOG,AMZN,TM,F,JPM
Symbols,,,,,,,,,,
AAPL,0.056962,0.026320,0.035018,0.035597,0.025481,0.034471,0.043662,0.015030,0.012213,0.019482
BA,0.026320,0.064442,0.028154,0.031532,0.028079,0.028681,0.034109,0.017858,0.014637,0.023519
TSLA,0.035018,0.028154,0.229940,0.041285,0.023631,0.034706,0.045234,0.012347,0.023461,0.019655
BABA,0.035597,0.031532,0.041285,0.100194,0.027906,0.043376,0.051852,0.017723,0.018471,0.020923
GS,0.025481,0.028079,0.023631,0.027906,0.052992,0.027371,0.028061,0.017244,0.022015,0.035589
GOOG,0.034471,0.028681,0.034706,0.043376,0.027371,0.051729,0.048904,0.018814,0.015602,0.020515
AMZN,0.043662,0.034109,0.045234,0.051852,0.028061,0.048904,0.086385,0.017119,0.014872,0.020684
TM,0.015030,0.017858,0.012347,0.017723,0.017244,0.018814,0.017119,0.026472,0.014425,0.014592
F,0.012213,0.014637,0.023461,0.018471,0.022015,0.015602,0.014872,0.014425,0.053109,0.020259


In [30]:
num = 10

In [31]:
mm=1
r0=1/24
r1=0.02*r0
rr=[r1]

num = 10
A1=Cov*r0
A2=np.zeros((num,mm))
A3=np.zeros((mm,num))
A4=np.diag(rr)

Omega1=np.concatenate((A1,A2),axis=1)
Omega2=np.concatenate((A3,A4),axis=1)
Omega=np.concatenate((Omega1,Omega2),axis=0)
Omega
# A1=np.insert(Cov*r0,-1)

array([[0.00237342, 0.00109666, 0.00145909, 0.00148323, 0.00106172,
        0.00143629, 0.00181924, 0.00062627, 0.00050889, 0.00081174,
        0.        ],
       [0.00109666, 0.00268507, 0.00117308, 0.00131383, 0.00116996,
        0.00119506, 0.00142122, 0.00074406, 0.00060987, 0.00097996,
        0.        ],
       [0.00145909, 0.00117308, 0.00958084, 0.00172022, 0.00098462,
        0.00144608, 0.00188477, 0.00051444, 0.00097755, 0.00081895,
        0.        ],
       [0.00148323, 0.00131383, 0.00172022, 0.00417473, 0.00116273,
        0.00180734, 0.00216052, 0.00073845, 0.00076964, 0.00087181,
        0.        ],
       [0.00106172, 0.00116996, 0.00098462, 0.00116273, 0.002208  ,
        0.00114047, 0.0011692 , 0.00071848, 0.00091728, 0.00148288,
        0.        ],
       [0.00143629, 0.00119506, 0.00144608, 0.00180734, 0.00114047,
        0.00215536, 0.00203767, 0.00078392, 0.00065009, 0.00085481,
        0.        ],
       [0.00181924, 0.00142122, 0.00188477, 0.00216052, 0.

$\Sigma^{BL}$

In [32]:
B1=np.eye(num,num)
P=P.reshape(1,num)
B=np.concatenate((B1,P),axis=0)
B.shape

(11, 10)

In [33]:
Cov2=np.dot(np.dot(B.T,np.linalg.inv(Omega)),B)
Cov2 = np.linalg.inv(Cov2)

In [34]:
Cov2

array([[0.0023539 , 0.0010673 , 0.0014573 , 0.0015143 , 0.00099731,
        0.00141657, 0.0017542 , 0.00062248, 0.00056816, 0.00077394],
       [0.0010673 , 0.0026409 , 0.00117039, 0.00136058, 0.00107306,
        0.00116538, 0.00132338, 0.00073837, 0.00069906, 0.0009231 ],
       [0.0014573 , 0.00117039, 0.00958068, 0.00172307, 0.00097873,
        0.00144427, 0.00187882, 0.00051409, 0.00098297, 0.00081549],
       [0.0015143 , 0.00136058, 0.00172307, 0.00412525, 0.00126529,
        0.00183875, 0.00226407, 0.00074447, 0.00067525, 0.00093199],
       [0.00099731, 0.00107306, 0.00097873, 0.00126529, 0.00199543,
        0.00107536, 0.00095458, 0.00070599, 0.00111291, 0.00135814],
       [0.00141657, 0.00116538, 0.00144427, 0.00183875, 0.00107536,
        0.00213542, 0.00197194, 0.0007801 , 0.00071001, 0.0008166 ],
       [0.0017542 , 0.00132338, 0.00187882, 0.00226407, 0.00095458,
        0.00197194, 0.00338268, 0.0007007 , 0.00081718, 0.00073588],
       [0.00062248, 0.00073837, 0.0005140

In [35]:
Cov_BL=Cov1+Cov2

In [36]:
Cov_BL

Symbols,AAPL,BA,TSLA,BABA,GS,GOOG,AMZN,TM,F,JPM
Symbols,,,,,,,,,,
AAPL,0.059316,0.027387,0.036475,0.037112,0.026478,0.035888,0.045416,0.015653,0.012781,0.020256
BA,0.027387,0.067083,0.029324,0.032892,0.029152,0.029847,0.035433,0.018596,0.015336,0.024442
TSLA,0.036475,0.029324,0.239521,0.043008,0.024610,0.036150,0.047113,0.012861,0.024444,0.020470
BABA,0.037112,0.032892,0.043008,0.104319,0.029171,0.045215,0.054116,0.018467,0.019147,0.021855
GS,0.026478,0.029152,0.024610,0.029171,0.054987,0.028447,0.029015,0.017950,0.023128,0.036947
GOOG,0.035888,0.029847,0.036150,0.045215,0.028447,0.053864,0.050876,0.019594,0.016312,0.021332
AMZN,0.045416,0.035433,0.047113,0.054116,0.029015,0.050876,0.089768,0.017820,0.015689,0.021420
TM,0.015653,0.018596,0.012861,0.018467,0.017950,0.019594,0.017820,0.027574,0.015038,0.015193
F,0.012781,0.015336,0.024444,0.019147,0.023128,0.016312,0.015689,0.015038,0.055141,0.021218


In [37]:
# B1=np.eye(1,1)
# P=P.reshape(-1,1)
# B=np.concatenate((B1,P),axis=0)
# # B.shape
# # Cov2=np.dot(np.dot(B.T,np.linalg.inv(Omega)),B)
# # Cov2

$\mu^{BL}$

In [38]:
C3=np.concatenate((r_hat,q),axis=0)
C3

array([ 0.11651814,  0.08798639,  0.11975029,  0.13241186,  0.08287265,
        0.11585523,  0.14803261,  0.05670152,  0.05320767,  0.06778732,
       -0.0133348 ])

In [39]:
C1=np.dot(np.linalg.inv(Cov2),B.T)
C2=np.dot(np.linalg.inv(Omega),C3)
mu_BL=np.dot(C1,C2)

C1=np.dot(np.linalg.inv(Cov2),B.T)
C2=np.dot(Omega,C3)
mu_BL=np.dot(C1,C2)

C1=np.dot(Cov2,B.T)
C2=np.dot(np.linalg.inv(Omega),C3)
mu_BL=np.dot(C1,C2)

In [40]:
mu_BL

array([0.11651814, 0.08798639, 0.11975029, 0.13241186, 0.08287265,
       0.11585523, 0.14803261, 0.05670152, 0.05320767, 0.06778732])

In [41]:
mu

Symbols
AAPL    0.169469
BA      0.385788
TSLA    0.215126
BABA    0.219486
GS     -0.172359
GOOG    0.138631
AMZN    0.346854
TM      0.021575
F      -0.196012
JPM     0.082235
dtype: float64

#### BL result

In [42]:
num

10

In [43]:
def opt1(mu,C,rf):
    
    e=np.zeros(num)+1
    
    def markowitz(weights, mu, C, rf):
    
        weights=np.array(weights)
    
        obj_func = np.dot(mu.T,weights) +  (1-np.dot(e.T,weights))*rf
    
#         print (-obj_func,weights)
        return -obj_func

        
    cons = ({'type':'eq', 'fun':lambda x: np.sum(x)-1},
       {'type':'ineq', 'fun':lambda x: -np.sqrt(np.dot(np.dot(x.T,C),x))+L})

    #No short constraint
#     bnds = [(0,1)]*num
    weights = np.ones(num)/num
    res = scipy.optimize.minimize(markowitz, weights, (mu, C, rf), method = 'SLSQP',  constraints = cons,options={'maxiter':10000})
    print(res)
    if not res.success: raise BaseException(res.message)
    return res

In [44]:
opt1(mu_BL,Cov_BL,rf)

     fun: -0.4225501732940823
     jac: array([-0.10231746, -0.0737857 , -0.10554961, -0.11821118, -0.06867196,
       -0.10165454, -0.13383194, -0.04250083, -0.03900699, -0.05358664])
 message: 'Optimization terminated successfully.'
    nfev: 372
     nit: 31
    njev: 31
  status: 0
 success: True
       x: array([ 1.0775885 , -0.08416369,  0.14103334,  0.7889395 ,  0.71358406,
        1.15308114,  1.7338777 , -2.62436893, -0.76943764, -1.13013399])


     fun: -0.4225501732940823
     jac: array([-0.10231746, -0.0737857 , -0.10554961, -0.11821118, -0.06867196,
       -0.10165454, -0.13383194, -0.04250083, -0.03900699, -0.05358664])
 message: 'Optimization terminated successfully.'
    nfev: 372
     nit: 31
    njev: 31
  status: 0
 success: True
       x: array([ 1.0775885 , -0.08416369,  0.14103334,  0.7889395 ,  0.71358406,
        1.15308114,  1.7338777 , -2.62436893, -0.76943764, -1.13013399])

In [45]:

w = opt1(mu_BL,Cov_BL,rf).x *100
w = list(map(lambda x:round(x, 3), w))
print (w)

     fun: -0.4225501732940823
     jac: array([-0.10231746, -0.0737857 , -0.10554961, -0.11821118, -0.06867196,
       -0.10165454, -0.13383194, -0.04250083, -0.03900699, -0.05358664])
 message: 'Optimization terminated successfully.'
    nfev: 372
     nit: 31
    njev: 31
  status: 0
 success: True
       x: array([ 1.0775885 , -0.08416369,  0.14103334,  0.7889395 ,  0.71358406,
        1.15308114,  1.7338777 , -2.62436893, -0.76943764, -1.13013399])
[107.759, -8.416, 14.103, 78.894, 71.358, 115.308, 173.388, -262.437, -76.944, -113.013]


### MVIO

In [46]:
from cvxpy import *

In [47]:
mm=1
r0=1/24
r1=0.02*r0
rr=[r1]

num = 10
A1=Cov*r0
A2=np.zeros((num,mm))
A3=np.zeros((mm,num))
A4=np.diag(rr)

Omega1=np.concatenate((A1,A2),axis=1)
Omega2=np.concatenate((A3,A4),axis=1)
Omega=np.concatenate((Omega1,Omega2),axis=0)
Omega.shape
# A1=np.insert(Cov*r0,-1)

(11, 11)

In [53]:
##### special case with sigma = 0.5 * delta * sigma : BL
import cvxpy as cvx
mu_io = cvx.Variable((10,1))
sigma_bar = cvx.Variable((10,10))
t = cvx.Variable((1,1))
u = cvx.Variable((11,1))
rf = rf
Omega = Omega 
x_mkt = np.array(mkt).reshape(-1,1)
# x = cp.Variable((10,1))
e = e.reshape(-1,1)
I = np.identity(11)
q = q.reshape(-1,1)


tmp = 1.25*2*np.array(Cov).dot(np.array(x_mkt).reshape(-1,1))
# sigma_bar.dot(x_mkt)
Cons = [
    u - np.linalg.inv(np.sqrt(Omega)) @ cvx.vstack([mu_io - rf*e - tmp, \
                                                          P@mu_io - q]) == 0,
    cvx.vstack([ cvx.hstack([t, u.T]), cvx.hstack([u, I]) ]) >> 0,
    sigma_bar >= 0
]
objective = cvx.Minimize(t)
prob = cvx.Problem(objective, Cons)
prob.solve(solver='CVXOPT')  # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
# print("optimal var", x.value, y.value)
print(mu_io.value)


status: optimal
optimal value -2.0491421059976034e-17
[[0.11651814]
 [0.08798639]
 [0.11975029]
 [0.13241186]
 [0.08287265]
 [0.11585523]
 [0.14803261]
 [0.05670152]
 [0.05320767]
 [0.06778732]]


In [49]:
##### general case with sigma as vriable.
import cvxpy as cvx
mu_io = cvx.Variable((10,1))
sigma_bar = cvx.Variable((10,10))
t = cvx.Variable((1,1))
u = cvx.Variable((11,1))
rf = rf
Omega = Omega 
x_mkt = np.array(mkt).reshape(-1,1)
e = e.reshape(-1,1)
I = np.identity(11)
q = q.reshape(-1,1)

# sigma_bar.dot(x_mkt)
Cons = [
    u - np.linalg.inv(np.sqrt(Omega)) @ cvx.vstack([mu_io - rf*e - sigma_bar@x_mkt, \
                                                          P @ mu_io - q]) == 0,
    cvx.vstack([ cvx.hstack([t, u.T]), cvx.hstack([u, I]) ]) >> 0,
    sigma_bar >= 0
]

objective = cvx.Minimize(t)
prob = cvx.Problem(objective, Cons)
prob.solve(solver='CVXOPT')  # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
# print("optimal var", x.value, y.value)
print(mu_io.value)
print(sigma_bar.value)

status: optimal
optimal value -3.7786285416747996e-10
[[0.56484187]
 [0.56760807]
 [0.56760806]
 [0.56209261]
 [0.578844  ]
 [0.56484187]
 [0.57600918]
 [0.56760805]
 [0.55664457]
 [0.57039129]]
[[0.54956637 0.55317311 0.55295904 0.55167549 0.55320401 0.55073735
  0.54977142 0.55285086 0.55333969 0.55249006]
 [0.55340735 0.55340735 0.55340735 0.55340735 0.55340735 0.55340735
  0.55340735 0.55340735 0.55340735 0.55340735]
 [0.55340735 0.55340735 0.55340735 0.55340735 0.55340735 0.55340735
  0.55340735 0.55340735 0.55340735 0.55340735]
 [0.54575288 0.55293898 0.55251111 0.54994922 0.55300076 0.54808064
  0.54616013 0.55229495 0.55327204 0.55157433]
 [0.5690492  0.55434529 0.5552043  0.56039088 0.55422144 0.5642211
  0.56819993 0.55563907 0.55367805 0.5570922 ]
 [0.54956637 0.55317311 0.55295904 0.55167549 0.55320401 0.55073735
  0.54977142 0.55285086 0.55333969 0.55249006]
 [0.56509674 0.55411065 0.5547545  0.55863655 0.5540178  0.56149749
  0.56446419 0.55508027 0.55361036 0.55616863]
 

In [58]:
Cov

Symbols,AAPL,BA,TSLA,BABA,GS,GOOG,AMZN,TM,F,JPM
Symbols,,,,,,,,,,
AAPL,0.056962,0.026320,0.035018,0.035597,0.025481,0.034471,0.043662,0.015030,0.012213,0.019482
BA,0.026320,0.064442,0.028154,0.031532,0.028079,0.028681,0.034109,0.017858,0.014637,0.023519
TSLA,0.035018,0.028154,0.229940,0.041285,0.023631,0.034706,0.045234,0.012347,0.023461,0.019655
BABA,0.035597,0.031532,0.041285,0.100194,0.027906,0.043376,0.051852,0.017723,0.018471,0.020923
GS,0.025481,0.028079,0.023631,0.027906,0.052992,0.027371,0.028061,0.017244,0.022015,0.035589
GOOG,0.034471,0.028681,0.034706,0.043376,0.027371,0.051729,0.048904,0.018814,0.015602,0.020515
AMZN,0.043662,0.034109,0.045234,0.051852,0.028061,0.048904,0.086385,0.017119,0.014872,0.020684
TM,0.015030,0.017858,0.012347,0.017723,0.017244,0.018814,0.017119,0.026472,0.014425,0.014592
F,0.012213,0.014637,0.023461,0.018471,0.022015,0.015602,0.014872,0.014425,0.053109,0.020259


# Robust Model MVIO

## 1. first solve my the L_mkt

In [59]:

alpha = 0.5
k = 5
epsilon = 0.1

sigma = np.array(Cov)
w, v = np.linalg.eig(sigma)

w = w[:k]
sum_of_w = 1/alpha * np.sum(w)
v = v[:k]

rf = rf
e = np.ones(10).reshape(-1,1)

r = cvx.Variable((10,1))

sigma_0 = cvx.Variable((10,10), nonneg=True)

x_mkt = x_mkt

x = r - rf * e

# Solving for L
Cons = [
    matrix_frac(x, sigma_0) <= 1,
    cvx.trace(sigma_0) <= sum_of_w
]
for i in range(k):
    Cons = Cons + [cvx.norm(sigma_0 @ v[i].reshape(-1,1) - w[i] * v[i].reshape(-1,1)) <= epsilon]

objective = cvx.Minimize( r.T @ x_mkt)
prob = cvx.Problem(objective, Cons)
prob.solve(solver='CVXOPT')  # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
# print("optimal var", x.value, y.value)
print(r.value)
L_mkt = -r.value.T.dot(x_mkt)[0][0]
L_mkt

status: optimal
optimal value -0.2824129023347122
[[-0.26583645]
 [ 0.00352759]
 [-0.01166402]
 [-0.14166809]
 [-0.1280051 ]
 [-0.42897682]
 [-0.38254552]
 [-0.05895652]
 [-0.00280482]
 [-0.18828525]]


0.2824129023347123

## 2. then solve for the optimal mu

In [51]:
alpha = 0.5
k = 5
epsilon = 0.1

sigma = np.array(Cov)
w, v = np.linalg.eig(sigma)

w = w[:k]
sum_of_w = 1/alpha * np.sum(w)
v = v[:k]

rf = rf
e = np.ones(10).reshape(-1,1)

P = P
q = q

mu_0 = cvx.Variable((10,1))
z = cvx.Variable((1,1))
t = cvx.Variable((1,1))
mu_eq = cvx.Variable((10,1))
sigma_0 = cvx.Variable((10,10), nonneg=True)

x_mkt = x_mkt


# Solving for L
Cons = [
    (mu_eq - rf * e).T @ x_mkt == z * L_mkt,
    cvx.vstack([cvx.hstack([sigma_0, mu_eq]), cvx.hstack([mu_eq.T, z])]) >= 0,
    cvx.trace(sigma_0) <= sum_of_w,
    cvx.matrix_frac(cvx.vstack([mu_0-rf*e-mu_eq, P @ mu_0 - q]), Omega) <= t,
]
for i in range(k):
    Cons = Cons + [cvx.norm(sigma_0 @ v[i].reshape(-1,1) - z * w[i] * v[i].reshape(-1,1)) <= epsilon]

objective = cvx.Minimize(t[0,0])
prob = cvx.Problem(objective, Cons)
prob.solve(solver='SCS')  # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
# print("optimal var", x.value, y.value)
print(mu_0.value)


status: optimal
optimal value -1.0475359662513901e-07
[[0.02708712]
 [0.0157586 ]
 [0.02070638]
 [0.02241175]
 [0.01855324]
 [0.01964929]
 [0.06385291]
 [0.01605797]
 [0.01423246]
 [0.01606613]]
